In [ ]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels as sm
import branca.colormap as cmp
import folium
import shapely
import plotly.express as px

In [ ]:
# Canada, Quebec City Dataframe (Main City)
#df, summary data frame
df=pd.read_csv('../data/canada/listings.csv')
#dfd, detailed data frame
dfd=pd.read_csv('../data/canada/detailed_listing.csv')

In [ ]:
# Remove coloumn 'neighbourhood_group' and 'license'  because it's all NaN
df.drop(['neighbourhood_group','license'], axis=1, inplace=True)
# Show the shape
df.shape

In [ ]:
# Sample 10 Rows (Summary)
df.head(10)

In [ ]:
# Sample 10 Rows (Detailed)
dfd.head(10)

In [ ]:
# Correlation heatmap for summary
df.corr().style.background_gradient(cmap='coolwarm')

In [ ]:
# Correlation heatmap for detailed
dfd.corr().style.background_gradient(cmap='coolwarm')

In [ ]:
# review score correlation
dfr = dfd.iloc[:,61:67]
dfr.corr()

In [ ]:
# 1/3 Sorts neighborhood mean price 
neig = df[['price','neighbourhood']].groupby('neighbourhood').mean()
neig = neig.sort_values('price')
neig['rank'] = np.arange(len(neig)) + 1
print(neig)

In [ ]:
# 2/3 only run once or it will keep on creating indexes
neig = neig.reset_index(level=0)


In [ ]:
# 3/3 The Column
print(neig)

In [ ]:
# Bar plot neighborhood mean sorted (draft v2)
plt.bar(*zip(*neig.sort_values(ascending=False).items())) + plt.xticks(rotation=90)
plt.show


In [ ]:
# Set bounds
BBox = ((df.longitude.min(),   df.longitude.max(), df.latitude.min(), df.latitude.max()))
# Import Image
ruh_m = plt.imread('../data/canada/map.png')
# Plot Data on Map (Draft v1)
fig, ax = plt.subplots(figsize = (8,7))
ax.scatter(df.longitude, df.latitude, zorder=1, alpha= 0.2, c='b', s=10)
ax.set_title('Plotting Spatial Data on Montreal Map')
ax.set_xlim(BBox[0],BBox[1])
ax.set_ylim(BBox[2],BBox[3])
ax.imshow(ruh_m, zorder=0, extent = BBox, aspect= 'equal')


In [ ]:
#linear = cmp.LinearColormap(['Blue', 'Red'], vmin = df.price.min(), vmax=df.price.max())

In [ ]:
# Bar plot of room types' mean price sorted (Working on)
plot3=sns.barplot(x='room_type', y='price',data=df)
plot3.set(xlabel='Room Type', ylabel='Average Price',title='Room Type Average Price in Montreal')
plt.show()